In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import sklearn as scipy
from math import log, e
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_fake =pd.read_csv('data/newsData/Fake.csv')
df_true =pd.read_csv('data/newsData/Fake.csv')

df_fake['target'] = 0
df_true['target'] = 1
df = df_true.append(df_fake, ignore_index=True)

df.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [3]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
ps = PorterStemmer()
def getTokenMatrix(text):
    # lower case applied to every char save puncuation
    text = [word.lower() for word in text if word not in string.punctuation]
    # turn array of lower case letters into words, delimitter is a space 
    text = ''.join(text)
    pattern = "[^\w]"
    deltaDf = re.sub(pattern, " ",  text).split()
    deltaDf = [ps.stem(word) for word in deltaDf if word not in 
               stopwords]
    return deltaDf

[nltk_data] Downloading package stopwords to /Users/kjams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df['features'] = df['title'].apply(lambda x: getTokenMatrix(x))
df['features'] = list(zip(df.features, df.target))
df = df.drop(['date', 'subject', 'target'], axis=1)
df.head()

,title,text,features
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,"([donald, trump, send, embarrass, new, year, e..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,"([drunk, brag, trump, staffer, start, russian,..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...","([sheriff, david, clark, becom, internet, joke..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...","([trump, obsess, even, obama, name, code, webs..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,"([pope, franci, call, donald, trump, christma,..."


In [5]:
class Node:
    # takes in a single row from one consistent pandas dataframe 
    # to make each row an object, so that KNN and ID3 can be exposed 
    # as an a RESTFUL API. 
    # methods: KNN, ID3 applied dynamicaly via input constraints
    # 
    def __init__(self, features, k=5):
        self.features  = features[0]
        self.m = len(features)
        self.k = k
        self.label = features[1]
        self.edges  = list()
        self.entropy  = self.entropy() 
        self.infoGain = 1 - self.entropy
#         self.classification = self.s1['target']
#         self.embeddings = self.s1['token']
        
# DONE Jaccard Index & Score
    def jaccard(self, endNode):
        # endNode -> Node2.row
        s1 = set(self.features)
        s2 = set(endNode.features)
        intersection = len(s1.intersection(s2))
        union = (len(s1) + len(s2)) - intersection
        jaccardSimularity = float(intersection) / union
        jaccardDistance = 1-jaccardSimularity
        return (jaccardSimularity,jaccardDistance)
    
    def entropy(self, base=None):
        # NP Array
        features = self.features
        value,counts = np.unique(features, return_counts=True)
        # normalize counts for self.row
        norm_counts = counts / counts.sum()
        # Base log e as default, or whatever else
        base = e if base is None else base
        # mathmatical formula for information entropy 
        res = -(norm_counts * np.log(norm_counts)/np.log(base)).sum()
        return res
      

In [10]:
df['obj'] = df['features'].apply(lambda x: Node(x))

In [7]:
n = df['obj'].iloc[1]
print(n)
n2 = df['obj'].iloc[1]
print(n2)
n.jaccard(n2)


(1.0, 0.0)

In [8]:
class Graph:
    
    def __init__(self, listOfNodes):
        self.nodes = listOfNodes
        self.dict = {idx:(node.entropy, node) for idx,node in enumerate(listOfNodes)}            
        self.n = len(self.nodes)
        self.createAdjMatrix = self.adj((0,0))
        
    def createAdjMatrix(self,encoding):
        # t = 0, 0:jaccard distance, 1:index, 2:euclidean
        # (0,t) = distance -> T
        # (1,k) = cluster groups together at K leaf

        pass
    
    def knn(self, origin):
        pass
       
    

In [9]:
# TODO D-Tree

